We will be accessing files on S3 in this notebook. Since those files are public, we set the `AWS_NO_SIGN_REQUEST` to tell `rasterio` to skip the sign-in.

In [ ]:
%env AWS_NO_SIGN_REQUEST=YES

In [ ]:
from rastervision.core.data import ClassConfig

class_config = ClassConfig(
    names=['Car', 'Building', 'Low Vegetation', 'Tree', 'Impervious', 'Clutter', 'null'],
    colors=['#ffff00', '#0000ff', '#00ffff', '#00ff00', '#ffffff', '#ff0000', '#000000'],
)

## Define model

Use a [light-weight panoptic FPN model](https://github.com/AdeelH/pytorch-fpn) with a ResNet-18 backbone.

In [ ]:
import torch

model = torch.hub.load(
    'AdeelH/pytorch-fpn:0.3',
    'make_fpn_resnet',
    name='resnet50',
    fpn_type='panoptic',
    num_classes=len(class_config),
    fpn_channels=256,
    in_channels=3,
    out_size=(256, 256),
    pretrained=False)

In [ ]:
model

In [ ]:
from torchvision.utils import make_grid
from matplotlib import pyplot as plt

first_conv_layer = model[0].m[0][0].weight.cpu()
grid = make_grid(first_conv_layer, normalize=True)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(grid.permute(1, 2, 0))
ax.axis('off')
plt.show()

## Load trained weights

In [ ]:
from rastervision.pipeline.file_system.utils import download_if_needed

In [ ]:
model_weights_uri = 's3://azavea-research-public-data/raster-vision/examples/model-zoo-0.30/isprs-potsdam-ss/train/last-model.pth'
model_weights_path = download_if_needed(model_weights_uri)
model.load_state_dict(torch.load(model_weights_path, map_location='cpu'))

In [ ]:
first_conv_layer = model[0].m[0][0].weight.cpu()
grid = make_grid(first_conv_layer, normalize=True)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(grid.permute(1, 2, 0))
ax.axis('off')
plt.show()